In [3]:
import pandas as pd
import numpy as np
ccafp=pd.read_csv('C:/Users/91868/Desktop/Dataset_Reducing_Commercial_Aviation_Fatalities.csv')

#Splitting the data into Train, Test Sets
from sklearn.model_selection import train_test_split

y=ccafp['event']
ccafp.drop(columns='event',inplace=True)
X_train,X_test,y_train,y_test=train_test_split(ccafp,y,test_size=0.3,random_state=50)


#need to de the Labelencode the Categorical columns
from sklearn.preprocessing import LabelEncoder

catcols=[ele for ele in ccafp.columns if ccafp[ele].dtype==object]

for col in catcols:
    le=LabelEncoder()
    #le.fit(X_train[col])
    X_train[col]=le.fit_transform(X_train[col])
    X_test[col]=le.transform(X_test[col])

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=8, min_samples_split=8,
                       min_weight_fraction_leaf=0.0,n_jobs=None, oob_score=True, random_state=100, verbose=0,
                       warm_start=False)
clf.fit(X_train,y_train)
print("train_Score=",clf.score(X_train,y_train))
print("test_Score=",clf.score(X_test,y_test))

D:\ANACONDA - PYTHON\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\ANACONDA - PYTHON\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\ANACONDA - PYTHON\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\ANACONDA - PYTHON\lib\site-packages\sklearn\ensemble\forest.py:460: UserWarning: Some inputs do no

train_Score= 0.9391214192876859
test_Score= 0.9390538921013565


In [9]:
from sklearn.model_selection import RandomizedSearchCV

grid1={'max_depth':[6,7,8,9,10],'min_samples_split':[3,4,5,6,7,8],
       'min_samples_leaf':[5,6,7,8,9]}

grid=RandomizedSearchCV(estimator=clf,param_distributions=grid1,n_iter=15,cv=5)
grid.fit(X_train,y_train)
grid.get_params
grid.cv_results_
print(grid.best_estimator_)
print(grid.best_score_)

grid.best_params_

D:\ANACONDA - PYTHON\lib\site-packages\sklearn\ensemble\forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
D:\ANACONDA - PYTHON\lib\site-packages\sklearn\ensemble\forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
D:\ANACONDA - PYTHON\lib\site-packages\sklearn\ensemble\forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
D:\ANACONDA - PYTHON\lib\site-packages\sklearn\ensemble\forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
D:\ANACONDA - PYTHON\lib\site-packages\sklearn\ensemble\forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees we

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=8, min_samples_split=8,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=True, random_state=50, verbose=0,
                       warm_start=False)
0.9396772760837164


D:\ANACONDA - PYTHON\lib\site-packages\sklearn\ensemble\forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
D:\ANACONDA - PYTHON\lib\site-packages\sklearn\ensemble\forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


{'min_samples_split': 8, 'min_samples_leaf': 8, 'max_depth': 10}

In [4]:
catcols

['experiment', 'event']

In [6]:
#AUC Score
y_pred_proba=clf.predict_proba(X_test)

from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
print(lb.fit(y_test))
print(lb.classes_)
y_test_new=lb.transform(y_test)

from sklearn.metrics import roc_auc_score
auc=roc_auc_score(y_test_new,y_pred_proba)
auc

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)
['A' 'B' 'C' 'D']


0.9797027053742297

In [7]:
#Evaluation Metrics of the above Multiclass Classification 
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
print(multilabel_confusion_matrix(y_test,clf.predict(X_test)))

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,clf.predict(X_test)))

print(classification_report(y_test,clf.predict(X_test)))

[[[126651  19085]
  [    87 168750]]

 [[307666      0]
  [  5212   1695]]

 [[190708     77]
  [     0 123788]]

 [[299522     10]
  [ 13873   1168]]]
[[168750      0     77     10]
 [  5212   1695      0      0]
 [     0      0 123788      0]
 [ 13873      0      0   1168]]
              precision    recall  f1-score   support

           A       0.90      1.00      0.95    168837
           B       1.00      0.25      0.39      6907
           C       1.00      1.00      1.00    123788
           D       0.99      0.08      0.14     15041

    accuracy                           0.94    314573
   macro avg       0.97      0.58      0.62    314573
weighted avg       0.94      0.94      0.92    314573

